#Importing Libraries

In [ ]:

from pyspark.sql import SparkSession
import numpy as np
import pyspark.sql.functions as f
from pyspark.sql.window import Window
import sys
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.ml.feature import *
from pyspark.ml import Pipeline
import random

In [ ]:
random.seed(10)

#Initialising Spark session

In [ ]:
spark = (SparkSession
           .builder
           .appName("MySpark")
           .getOrCreate())

In [ ]:
sc = spark.sparkContext

#Reading and Displaying the Data

In [ ]:
df = spark.read.csv("diabetic_data_valid.csv", inferSchema=True, header=True,nullValue="?")

In [ ]:
df.show()

+----------------+------------------+--------------+---------------+----------------+----------------+---------------+------+--------+-----------------+------------------------+-------------------+------+------+------+-------------+---------+---------+-----------+-----------+---------+---------+------------+-------------+-------+------+-----------+----------------------+----------+
|time_in_hospital|num_lab_procedures|num_procedures|num_medications|number_inpatient|number_diagnoses|           race|gender|     age|admission_type_id|discharge_disposition_id|admission_source_id|diag_1|diag_2|diag_3|max_glu_serum|A1Cresult|metformin|repaglinide|glimepiride|glipizide|glyburide|pioglitazone|rosiglitazone|insulin|change|diabetesMed|readmitted_multinomial|readmitted|
+----------------+------------------+--------------+---------------+----------------+----------------+---------------+------+--------+-----------------+------------------------+-------------------+------+------+------+------------

#Counting distinct values in each column

In [ ]:
df.select([f.countDistinct(f.col(c)).alias(c) for c in df.columns]).show()

+----------------+------------------+--------------+---------------+----------------+----------------+----+------+---+-----------------+------------------------+-------------------+------+------+------+-------------+---------+---------+-----------+-----------+---------+---------+------------+-------------+-------+------+-----------+----------------------+----------+
|time_in_hospital|num_lab_procedures|num_procedures|num_medications|number_inpatient|number_diagnoses|race|gender|age|admission_type_id|discharge_disposition_id|admission_source_id|diag_1|diag_2|diag_3|max_glu_serum|A1Cresult|metformin|repaglinide|glimepiride|glipizide|glyburide|pioglitazone|rosiglitazone|insulin|change|diabetesMed|readmitted_multinomial|readmitted|
+----------------+------------------+--------------+---------------+----------------+----------------+----+------+---+-----------------+------------------------+-------------------+------+------+------+-------------+---------+---------+-----------+-----------+--

#Changing the datatypes of required variables

In [ ]:
df = df.withColumn("admission_type_id",df.admission_type_id.cast("String"))
df = df.withColumn("discharge_disposition_id",df.discharge_disposition_id.cast("String"))
df = df.withColumn("admission_source_id",df.admission_source_id.cast("String"))

#Separating numeric, categorical and target variables

In [ ]:
num_variables = df.columns[:6]
cat_variables = df.columns[6:12] + df.columns[15:-2]
target = df.columns[-1]
#cat_variables

#Outlier Treatment on Numeric Variables

In [ ]:
for variable in num_variables:
    q1, q2, q3 = df.select(f.percentile_approx(variable, [0.25, 0.5, 0.75], 1000000).alias("quantiles")).collect()[0]["quantiles"]
    IQR = q3 - q1
    lower_value = q1 - 1.5 * IQR
    upper_value = q3 + 1.5 * IQR
    df = df.withColumn(variable,
                       f.when(f.col(variable) < lower_value,
                              lower_value
                              ).when(f.col(variable) > upper_value,
                                     upper_value
                                     ).otherwise(f.col(variable))
                        )

#Defining stages for model data preparation

In [ ]:
stages = []

for variable in cat_variables:

    # Index Categorical variable
    indexer = StringIndexer(inputCol=variable, outputCol=variable + "_index")
    
    # One Hot Encode Categorical variable
    encoder = OneHotEncoder(inputCols=[indexer.getOutputCol()],
                                     outputCols=[variable + "_OHE_vec"])
    # Append Pipeline Stages
    stages += [indexer, encoder]



# Scale Feature: Select the Features to Scale using helper 'select_features_to_scale' function above and Standardize 
assembler_num = VectorAssembler(inputCols=num_variables, outputCol="num_variables")
scaler_num = StandardScaler(inputCol="num_variables", outputCol="scaled_num_variables")
stages += [assembler_num,scaler_num]


# Assemble or Concat the Categorical Features Features
assembler_cat = [variable + "_OHE_vec" for variable in cat_variables]

assembler = VectorAssembler(inputCols=assembler_cat, outputCol="assembler_cat") 

stages += [assembler]

# Assemble Final Training Data of Scaled, Numeric, and Categorical Engineered Features
assembler_final = VectorAssembler(inputCols=["assembler_cat","scaled_num_variables"], outputCol="variables")

stages += [assembler_final]

# Index Target Variable
target_str_index =  StringIndexer(inputCol=target, outputCol="target")

stages += [target_str_index]

In [ ]:
stages

[StringIndexer_7fbb9939df7b,
 OneHotEncoder_b6cfa712eb20,
 StringIndexer_b07fac60f55e,
 OneHotEncoder_b6e38fdec9cb,
 StringIndexer_f2ebbab0ae29,
 OneHotEncoder_c7306286c004,
 StringIndexer_5c91150318a4,
 OneHotEncoder_7ed02a4f1eee,
 StringIndexer_511877c6f62d,
 OneHotEncoder_97820a08ad36,
 StringIndexer_2b8639584f18,
 OneHotEncoder_16111f0521ba,
 StringIndexer_6a72a5c06e5a,
 OneHotEncoder_91443b56f5ea,
 StringIndexer_aaa46c184b93,
 OneHotEncoder_ea201abccde9,
 StringIndexer_4e6be94dabe3,
 OneHotEncoder_44072a537b59,
 StringIndexer_9a3c4af918ee,
 OneHotEncoder_fb780aa09d42,
 StringIndexer_0d8bb1ec9940,
 OneHotEncoder_fe8e4803ad5c,
 StringIndexer_b17b78c037c1,
 OneHotEncoder_f28842b1dd19,
 StringIndexer_f1ea9bf3a2ce,
 OneHotEncoder_8425273d43dd,
 StringIndexer_c62b3f4d2ea8,
 OneHotEncoder_bdbaccf345ce,
 StringIndexer_47537d42d690,
 OneHotEncoder_34bb2da54ec7,
 StringIndexer_42abcb42bfc6,
 OneHotEncoder_fe038e990f4c,
 StringIndexer_20b64a6ef975,
 OneHotEncoder_d69af3eba582,
 StringIndexer

In [ ]:
# Set Pipeline
pipeline = Pipeline(stages=stages)

# Fit Pipeline to Data
pipeline_model = pipeline.fit(df)

# Transform Data using Fitted Pipeline
transf_df = pipeline_model.transform(df)

In [ ]:
transf_df.show()

+----------------+------------------+--------------+---------------+----------------+----------------+---------------+------+--------+-----------------+------------------------+-------------------+------+------+------+-------------+---------+---------+-----------+-----------+---------+---------+------------+-------------+-------+------+-----------+----------------------+----------+----------+-------------+------------+--------------+---------+-------------+-----------------------+-------------------------+------------------------------+--------------------------------+-------------------------+---------------------------+-------------------+---------------------+---------------+-----------------+---------------+-----------------+-----------------+-------------------+-----------------+-------------------+---------------+-----------------+---------------+-----------------+------------------+--------------------+-------------------+---------------------+-------------+---------------+-----

#Keeping the columns only with the prepared data

In [ ]:
model_df = transf_df[["variables","target"]]

#Spliting data into train and test

In [ ]:
# Split Data into Train / Test Sets
train_data, test_data = model_df.randomSplit([.7, .3],seed=1234)

In [ ]:
train_rdd = train_data.rdd

In [ ]:
test_rdd = test_data.rdd
n = test_rdd.count()

#Setting hyper-parameters

In [ ]:
m = train_rdd.count()
beta = sc.broadcast(np.array([random.random() for _ in range(103)]))
alpha = 0.1
lambda_ = 0.1

In [ ]:
beta.value

array([0.57140259, 0.42888905, 0.5780913 , 0.20609823, 0.81332125,
       0.82358887, 0.65347253, 0.16022956, 0.52066936, 0.32777281,
       0.24999668, 0.95281691, 0.99655699, 0.04455638, 0.86016104,
       0.60319061, 0.38160599, 0.28361822, 0.67496485, 0.45683115,
       0.68586149, 0.66184632, 0.13297814, 0.76783781, 0.98241325,
       0.96938816, 0.61332682, 0.04426063, 0.00405514, 0.13397253,
       0.94100227, 0.30286056, 0.3661456 , 0.89819624, 0.3143638 ,
       0.54898218, 0.43603096, 0.06499418, 0.58454623, 0.8440679 ,
       0.15641892, 0.22429897, 0.41287021, 0.03692487, 0.4966035 ,
       0.81798286, 0.65789094, 0.53348374, 0.85512574, 0.14968813,
       0.56723548, 0.37417482, 0.60130455, 0.1129157 , 0.7755127 ,
       0.09660583, 0.16636375, 0.80744975, 0.94769392, 0.4332315 ,
       0.41408083, 0.24548188, 0.27470339, 0.61725277, 0.17846041,
       0.1194825 , 0.45573486, 0.16493781, 0.64872483, 0.82105567,
       0.77751139, 0.48013339, 0.34757126, 0.43471055, 0.00500

#Defining functions for Training the model

In [ ]:
def l_beta(x):
    x_ = np.insert(x["variables"].toArray(),0,1.0)
    beta_x = np.dot(x_,beta.value)
    sig = 1/(1 + np.exp(-beta_x))
    log1 = np.log(sig)
    log2 = np.log(1 - sig)
    y = x["target"]
    loss = y*log1 + (1-y)*log2
    return -loss

def grad(x):
    x_ = np.insert(x["variables"].toArray(),0,1.0)
    beta_x = np.dot(x_,beta.value)
    sig = 1/(1 + np.exp(-beta_x))
    y = x["target"]
    y_h = sig - y
    return x_ * y_h

#Model Training 

In [ ]:
#beta_x = X.map(betaT_x)
#h_x = beta_x.map(sigmoid)
#log_h_x = h_x.map(log)
for i in range(50):
    L = train_rdd.map(l_beta)
    train_loss = (L.reduce(lambda x,y: x+y) + lambda_ * np.dot(beta.value[1:],beta.value[1:])/2)/m
    L_test = test_rdd.map(l_beta)
    test_loss = (L.reduce(lambda x,y: x+y))/n
    print("Iteration",i+1," Train_Loss ->",train_loss," Test_loss ->",test_loss)
    del_beta =  train_rdd.map(grad)
    d_beta = del_beta.reduce(lambda x,y: x+y)
    beta = sc.broadcast(beta.value - alpha *(d_beta + lambda_ * np.insert(beta.value[1:],0,0)) / m)
    #print(beta.value[0:5])

Iteration 1  Train_Loss -> 6.067515433853908  Test_loss -> 14.133533047933566
Iteration 2  Train_Loss -> 4.957454356300818  Test_loss -> 11.547774475189714
Iteration 3  Train_Loss -> 3.8490080805695497  Test_loss -> 8.965777044501404
Iteration 4  Train_Loss -> 2.753810886260234  Test_loss -> 6.414641478130133
Iteration 5  Train_Loss -> 1.7600670119731434  Test_loss -> 4.0998294573097525
Iteration 6  Train_Loss -> 1.1878682195984123  Test_loss -> 2.766958155963284
Iteration 7  Train_Loss -> 1.0748192003713248  Test_loss -> 2.5036235808505523
Iteration 8  Train_Loss -> 1.0574481500766946  Test_loss -> 2.4631598779245785
Iteration 9  Train_Loss -> 1.047548883103775  Test_loss -> 2.4401008390660253
Iteration 10  Train_Loss -> 1.0384226035579436  Test_loss -> 2.4188423834232355
Iteration 11  Train_Loss -> 1.029552683878488  Test_loss -> 2.398181085315297
Iteration 12  Train_Loss -> 1.0208954447268745  Test_loss -> 2.3780151985045292
Iteration 13  Train_Loss -> 1.0124451368863028  Test_loss 

#Prediction and Test Results 

In [ ]:
def metric_cal(x):
    x_ = np.insert(x["variables"].toArray(),0,1.0)
    beta_x = np.dot(x_,beta.value)
    sig = 1/(1 + np.exp(-beta_x))
    y = x["target"]
    y_hat = 1 if sig >= cuttoff.value else 0
    acc = 1 if y == y_hat else 0
    pre_num = 1 if y == y_hat and y_hat == 1 else 0
    pre_denom = 1 if y_hat == 1 else 0
    rec_num = 1 if y == y_hat and y == 1 else 0
    rec_denom = 1 if y == 1 else 0
    return np.array([acc, pre_num, pre_denom, rec_num, rec_denom])

In [28]:
cuttoff = sc.broadcast(0.5)
metrics = test_rdd.map(metric_cal)
cal_metrcis = metrics.reduce(lambda x,y: x+y)
Acc = float(cal_metrcis[0]/n )
Precision = float(cal_metrcis[1]/cal_metrcis[2] )
Recall = float(cal_metrcis[3]/cal_metrcis[4])
F1 = (2 * Precision * Recall)/(Precision + Recall)
print("Cutoff -> {:.2f} Acc -> {:.2f} Precision -> {:.2f} Recall -> {:.2f} F1 -> {:.2f}".format(cuttoff.value,Acc, Precision, Recall,F1))

Cutoff -> 0.50 Acc -> 0.49 Precision -> 0.45 Recall -> 0.45 F1 -> 0.45
